# Import Model

In [3]:
import tensorflow as tf

#tf.enable_eager_execution()

from tensorflow.python.keras.applications.vgg19 import VGG19

In [ ]:
model = VGG19(
    include_top =False,
    weights = 'imagenet'
)

model.trainable = False
model.summary()

66396160/80134624 [=======================>......] - ETA: 10:40

# Import Libraries and Helper Function

In [ ]:
from tensorflow.python.keras.image import load_img, img_to_array
from tensorflow.python.keras.appliactions.vgg19 import preprocess_input
from tensorflow.python.keras.models import Model

import numpy as np
import matplotlib.pylot as plt

%matplotlib inline

# Image Processing and Display

In [ ]:
def load_and_process_image(image_path):
    img = load_img(image_path)
    img = img_to_array(img)
    img = preprocess_input(img)      # to make image compatible with our VGG model
    img = np.expand_dims(img, axis =0)    # to convert to it to 4 dimensional
    return img

In [ ]:
def deprocess(x):
    #Opposite of preprocess_input()
    x[:,:,0] += 103.939
    x[:,:,1] += 116.779
    x[:,:,2] += 123.68
    x = x[:,:,::-1]     # change order of channels
    x = np.clip(x, 0, 255).astype('uint8')    # scale values between 0 to 255
    return x

def display_image(image):
    if len(image.shape) == 4:
        img = np.squeeze(image, axis = 0)   # opposite of expand_dims()
    img = deprocess(img)
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img)
    return

In [ ]:
#Content Image
display_image(load_and_process_image('profile_white_background.jpeg'))

In [ ]:
# Style Image
display_image(load_and_process_image('style.jpeg'))

# Content and style Models

In [ ]:
content_layer = 'block5_conv2'       # deep layer

# shallow layer
style_layers = [
    'block1_conv1',
    'block3_conv1',
    'block5_conv1'
]

content_Model = Model(
    inputs = model.input,
    outputs = model.get_layer(content_layer)
)

style_models = [Model(inputs = model.input,
                     outputs = model.get_layer(layer).output for layer in style_layers)]

# Content Cost

In [ ]:
def content_cost(content, generated):
    a_C = content_model(content)
    a_G = content_model(generated)
    cost = tf.reduce_mean(tf.sqaure(a_C -a_G))    # mean square difference
    return cost